In [2]:
import breizhcrops as bzh
from breizhcrops import BreizhCrops
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tqdm
from tqdm import tqdm
import torch
from torch.utils.data import Dataset, DataLoader
from torch import nn
from torch.optim import Adam
from sklearn.metrics import accuracy_score, cohen_kappa_score, f1_score, recall_score, precision_score
import os

In [3]:
frh01 = BreizhCrops("frh01")
frh02 = BreizhCrops("frh02")
frh03 = BreizhCrops("frh03")
frh04 = BreizhCrops("frh04")
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")

In [90]:
frh01.domain = 1
frh02.domain = 2
frh03.domain = 3
frh04.domain = 4

## Dummy part

I was using this code but it become useless at the end. However, I dont want to delete it(I may require it)

In [ ]:
import torch
from torch.utils.data import Dataset

class DatasetWrapper(Dataset):
    def __init__(self, original_dataset, domain):
        self.original_dataset = original_dataset
        self.domain = domain
    def __len__(self):
        return len(self.original_dataset)

    def __getitem__(self, index):
        # Get the original tuple from the dataset
        data, label, id = self.original_dataset[index]

        # Return a new tuple that includes the domain
        return (data, label, id, self.domain)

frh01 = DatasetWrapper(frh01, 1)
frh02 = DatasetWrapper(frh02, 2)
frh03 = DatasetWrapper(frh03, 3)
frh04 = DatasetWrapper(frh04, 4)


In [4]:
def split_dataset(dataset, X=0.8):
    
    total_samples = len(dataset)

    num_test_samples = int(X * total_samples)

    np.random.seed(42)

    test_indices = np.random.choice(total_samples, num_test_samples, replace=False)

    # Find complementary indices
    train_indices = np.setdiff1d(np.arange(total_samples), test_indices)

    test = torch.utils.data.Subset(dataset, test_indices)
    train = torch.utils.data.Subset(dataset, train_indices)
    return train, test

## Different get_loader functions to perform training and tests upon different combinastions of datasets

I write a more modular one function to handle all dataset combinations but I face many errors at different stages of codes in implementation step. Therefore, I turn to more 'manuel' way.

### 234

In [5]:
def get_dataloader(batchsize, X=0.8, num_workers=0, upperPerformance=False):

    frh01.domain = 3
    frh01_train, frh01_test = split_dataset(frh01, X=X)

    if upperPerformance:
        traindatasets = frh01_train
    else:
        frh02.domain = 0
        frh03.domain = 1
        frh04.domain = 2
        traindatasets = torch.utils.data.ConcatDataset([frh02, frh03, frh04])

    testdataset = frh01_test
    

    traindataloader = DataLoader(traindatasets, batch_size=batchsize, shuffle=True, num_workers=num_workers)
    testdataloader = DataLoader(testdataset, batch_size=batchsize, shuffle=False, num_workers=num_workers)

    meta = dict(
        ndims=13,
        num_classes=9,
        sequencelength=45
    )

    return traindataloader, testdataloader, meta

### 123

In [6]:
def get_dataloader123(batchsize, X=0.8, num_workers=0, upperPerformance=False):

    frh04.domain = 3
    frh04_train, frh04_test = split_dataset(frh04, X=X)

    if upperPerformance:
        traindatasets = frh04_train
    else:
        frh01.domain = 0
        frh02.domain = 1
        frh03.domain = 2
        traindatasets = torch.utils.data.ConcatDataset([frh01, frh02, frh03])

    testdataset = frh04_test
    

    traindataloader = DataLoader(traindatasets, batch_size=batchsize, shuffle=True, num_workers=num_workers)
    testdataloader = DataLoader(testdataset, batch_size=batchsize, shuffle=False, num_workers=num_workers)

    meta = dict(
        ndims=13,
        num_classes=9,
        sequencelength=45
    )

    return traindataloader, testdataloader, meta

### 124

In [7]:
def get_dataloader124(batchsize, X=0.8, num_workers=0, upperPerformance=False):

    frh03.domain = 3
    frh03_train, frh03_test = split_dataset(frh03, X=X)

    if upperPerformance:
        traindatasets = frh03_train
    else:
        frh01.domain = 0
        frh02.domain = 1
        frh04.domain = 2
        traindatasets = torch.utils.data.ConcatDataset([frh01, frh02, frh04])

    testdataset = frh03_test
    

    traindataloader = DataLoader(traindatasets, batch_size=batchsize, shuffle=True, num_workers=num_workers)
    testdataloader = DataLoader(testdataset, batch_size=batchsize, shuffle=False, num_workers=num_workers)

    meta = dict(
        ndims=13,
        num_classes=9,
        sequencelength=45
    )

    return traindataloader, testdataloader, meta

In [8]:
frh01.domain = 0
frh02.domain = 1
frh04.domain = 2
traindatasets = torch.utils.data.ConcatDataset([frh01, frh02, frh04])

### 134

In [9]:
def get_dataloader134(batchsize, X=0.8, num_workers=0, upperPerformance=False):

    frh02.domain = 3
    frh02_train, frh02_test = split_dataset(frh02, X=X)

    if upperPerformance:
        traindatasets = frh02_train
    else:
        frh01.domain = 0
        frh03.domain = 1
        frh04.domain = 2
        traindatasets = torch.utils.data.ConcatDataset([frh01, frh03, frh04])

    testdataset = frh02_test
    

    traindataloader = DataLoader(traindatasets, batch_size=batchsize, shuffle=True, num_workers=num_workers)
    testdataloader = DataLoader(testdataset, batch_size=batchsize, shuffle=False, num_workers=num_workers)

    meta = dict(
        ndims=13,
        num_classes=9,
        sequencelength=45
    )

    return traindataloader, testdataloader, meta

## Class definitions, train-test loops and etc

In [10]:
from torch.autograd import Function

class GradientReversalFunction(Function):
    @staticmethod
    def forward(ctx, x, lambda_):
        ctx.lambda_ = lambda_
        return x.view_as(x)

    @staticmethod
    def backward(ctx, grad_output):
        return (grad_output.neg() * ctx.lambda_, None)

class GradientReversalLayer(nn.Module):
    def __init__(self, lambda_=1.0):
        super(GradientReversalLayer, self).__init__()
        self.lambda_ = lambda_

    def set_lambda(self, lambda_):
        self.lambda_ = lambda_
        
    def forward(self, x):
        return GradientReversalFunction.apply(x, self.lambda_)
    
    

class LSTM_DANN(torch.nn.Module):
    def __init__(self, input_size=13, hidden_size=128, num_layers=4, output_size=9, bidirectional=False, dropout=0.2, c_norm=True, num_domains=3):
        super(LSTM_DANN, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.output_size = output_size
        self.c_norm = c_norm
        
        if num_layers > 1 and dropout > 0:
            self.dropout = dropout
        else:
            self.dropout = 0.0 
            
        self.clayernorm = nn.LayerNorm((hidden_size + hidden_size * bidirectional) * num_layers)
        
        self.lstm = nn.LSTM(input_size=input_size, hidden_size=hidden_size, num_layers=num_layers,
                            bias=False, batch_first=True, dropout=dropout, bidirectional=bidirectional)   
        
        
        if bidirectional:
            hidden_size = hidden_size * 2
            
        self.gradient_reversal = GradientReversalLayer(lambda_=1.0)
        self.label_classifyer = nn.Linear(hidden_size * num_layers, output_size, bias=True)
        self.domain_classifyer = nn.Linear(hidden_size * num_layers, num_domains, bias=True)
        
    def forward(self, x, lambda_=1.0):
        
        outputs, last_state_list = self.lstm.forward(x)

        h, c = last_state_list

        nlayers, batchsize, n_hidden = c.shape
        
        if self.c_norm:
            features = self.clayernorm(c.transpose(0, 1).contiguous().view(batchsize, nlayers * n_hidden))
        else:
            features = c.transpose(0, 1).contiguous().view(batchsize, nlayers * n_hidden)
            
        self.gradient_reversal.set_lambda(lambda_) 
        
        reverse_features = self.gradient_reversal(features)

        label_prediction = self.label_classifyer(features)
        domain_prediction = self.domain_classifyer(reverse_features)
        
        return label_prediction, domain_prediction
        
    def save(self, path="model.pth", **kwargs):
        print("\nsaving model to " + path)
        model_state = self.state_dict()
        os.makedirs(os.path.dirname(path), exist_ok=True)
        torch.save(dict(model_state=model_state, **kwargs), path)
        

In [12]:
def metrics(y_true, y_pred):
    accuracy = accuracy_score(y_true, y_pred)
    kappa = cohen_kappa_score(y_true, y_pred)
    f1_micro = f1_score(y_true, y_pred, average="micro")
    f1_macro = f1_score(y_true, y_pred, average="macro")
    f1_weighted = f1_score(y_true, y_pred, average="weighted")
    recall_micro = recall_score(y_true, y_pred, average="micro")
    recall_macro = recall_score(y_true, y_pred, average="macro")
    recall_weighted = recall_score(y_true, y_pred, average="weighted")
    precision_micro = precision_score(y_true, y_pred, average="micro")
    precision_macro = precision_score(y_true, y_pred, average="macro")
    precision_weighted = precision_score(y_true, y_pred, average="weighted")

    return dict(
        accuracy=accuracy,
        kappa=kappa,
        f1_micro=f1_micro,
        f1_macro=f1_macro,
        f1_weighted=f1_weighted,
        recall_micro=recall_micro,
        recall_macro=recall_macro,
        recall_weighted=recall_weighted,
        precision_micro=precision_micro,
        precision_macro=precision_macro,
        precision_weighted=precision_weighted,
    )


def train_epoch(model, optimizer, criterion_label, criterion_domain, dataloader, device, alpha=1.0):
    model.train()
    losses = list()
    label_losses = list()
    domain_losses = list()
    with tqdm(enumerate(dataloader), total=len(dataloader), leave=True) as iterator:
        for idx, batch in iterator:
            optimizer.zero_grad()
            x, y_true, _ , y_domain= batch
            prediction_label, prediction_domain = model.forward(x.to(device), lambda_=alpha)
            loss_label = criterion_label(prediction_label, y_true.to(device))
            loss_domain = criterion_domain(prediction_domain, y_domain.to(device))
            if alpha == 0:
                loss = loss_label
            else:
                #loss = loss_label + alpha * loss_domain
                loss = loss_label + loss_domain

            loss.backward()
            optimizer.step()
            iterator.set_description(f"train loss={loss_label:.2f}")
            losses.append(loss)
            label_losses.append(loss_label)
            domain_losses.append(loss_domain)
    return torch.stack(losses), torch.stack(label_losses), torch.stack(domain_losses)


def test_epoch(model, criterion, dataloader, device):
    model.eval()
    with torch.no_grad():
        losses = list()
        y_true_list = list()
        y_pred_list = list()
        field_ids_list = list()
        with tqdm(enumerate(dataloader), total=len(dataloader), leave=True) as iterator:
            for idx, batch in iterator:
                x, y_true, field_id, _ = batch
                logits = model.forward(x.to(device))[0]
                loss = criterion(logits, y_true.to(device))
                iterator.set_description(f"test loss={loss:.2f}")
                losses.append(loss)
                y_true_list.append(y_true)
                y_pred_list.append(logits.argmax(-1))
                field_ids_list.append(field_id)
        return torch.stack(losses), torch.cat(y_true_list), torch.cat(y_pred_list), torch.cat(field_ids_list)


# TESTS

In [ ]:
device = "mps" if torch.backends.mps.is_available() else "cpu"
model = LSTM_DANN(input_size=13, hidden_size=64, num_layers=2, output_size=9, bidirectional=False, dropout=0.2, c_norm=True, num_domains=3).to(device)
train_dataloader, test_dataloader, meta = get_dataloader(batchsize=32, num_workers=0, X=0.8, upperPerformance=False)
criterion_label = torch.nn.CrossEntropyLoss(reduction="mean")
criterion_domain = torch.nn.CrossEntropyLoss(reduction="mean")
optimizer = Adam(model.parameters(), 0.001, weight_decay=1e-6)
log = list()
n_epoch = 10
for epoch in range(n_epoch):
    p = float(epoch) / n_epoch 
    alpha = (2. / (1. + np.exp(-10 * p)) - 1)
    train_loss, label_loss, domain_loss = train_epoch(model, optimizer, criterion_label, criterion_domain, train_dataloader, device, alpha=0.05)
    test_loss, y_true, y_pred, *_ = test_epoch(model, criterion_label, test_dataloader, device)
    scores1 = metrics(y_true.cpu(), y_pred.cpu())
    scores_msg = ", ".join([f"{k}={v:.2f}" for (k, v) in scores1.items()])
    test_loss = test_loss.cpu().detach().numpy().mean()
    train_loss = train_loss.cpu().detach().numpy().mean()
    print(f"epoch {epoch}: trainloss {train_loss:.2f}, testloss {test_loss:.2f} " + scores_msg)

    scores1["epoch"] = epoch
    scores1["trainloss"] = train_loss
    scores1["testloss"] = test_loss
    scores1["labelloss"] = label_loss.cpu().detach().numpy().mean()
    scores1["domainloss"] = domain_loss.cpu().detach().numpy().mean()
    log.append(scores1)
    
    log_df = pd.DataFrame(log).set_index("epoch")
    log_df.to_csv("DANN234.csv")

In [14]:
device = "mps" if torch.backends.mps.is_available() else "cpu"
model = LSTM_DANN(input_size=13, hidden_size=64, num_layers=2, output_size=9, bidirectional=False, dropout=0.2, c_norm=True, num_domains=3).to(device)
train_dataloader, test_dataloader, meta = get_dataloader123(batchsize=32, num_workers=0, X=0.8, upperPerformance=False)
criterion_label = torch.nn.CrossEntropyLoss(reduction="mean")
criterion_domain = torch.nn.CrossEntropyLoss(reduction="mean")
optimizer = Adam(model.parameters(), 0.001, weight_decay=1e-6)
log = list()
n_epoch = 15
for epoch in range(n_epoch):
    """if epoch < 2:
    alpha = 0.0
else:
    p = float(epoch-2) / (n_epoch-2) 
    #alpha = (2. / (1. + np.exp(-5 * p)) - 1) * 0.4 + 0.1
    alpha = 0.1 + 0.4 * p"""
    p = float(epoch) / n_epoch
    alpha = (2. / (1. + np.exp(-10 * p)) - 1)
    train_loss, label_loss, domain_loss = train_epoch(model, optimizer, criterion_label, criterion_domain, train_dataloader, device, alpha=alpha)
    test_loss, y_true, y_pred, *_ = test_epoch(model, criterion_label, test_dataloader, device)
    scores1 = metrics(y_true.cpu(), y_pred.cpu())
    scores_msg = ", ".join([f"{k}={v:.2f}" for (k, v) in scores1.items()])
    test_loss = test_loss.cpu().detach().numpy().mean()
    train_loss = train_loss.cpu().detach().numpy().mean()
    print(f"epoch {epoch}: trainloss {train_loss:.2f}, testloss {test_loss:.2f} " + scores_msg)

    scores1["epoch"] = epoch
    scores1["trainloss"] = train_loss
    scores1["testloss"] = test_loss
    scores1["labelloss"] = label_loss.cpu().detach().numpy().mean()
    scores1["domainloss"] = domain_loss.cpu().detach().numpy().mean()
    log.append(scores1)

    log_df = pd.DataFrame(log).set_index("epoch")
    log_df.to_csv("DANN123.csv")

test loss=0.71: 100%|██████████| 3066/3066 [02:17<00:00, 22.25it/s]
/Users/aliyilmaz/anaconda3/envs/pytorch_env/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/aliyilmaz/anaconda3/envs/pytorch_env/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


epoch 0: trainloss 0.71, testloss 0.62 accuracy=0.74, kappa=0.65, f1_micro=0.74, f1_macro=0.48, f1_weighted=0.71, recall_micro=0.74, recall_macro=0.51, recall_weighted=0.74, precision_micro=0.74, precision_macro=0.49, precision_weighted=0.73


test loss=1.00: 100%|██████████| 3066/3066 [02:21<00:00, 21.67it/s]
/Users/aliyilmaz/anaconda3/envs/pytorch_env/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/aliyilmaz/anaconda3/envs/pytorch_env/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


epoch 1: trainloss 1.65, testloss 0.53 accuracy=0.76, kappa=0.69, f1_micro=0.76, f1_macro=0.52, f1_weighted=0.74, recall_micro=0.76, recall_macro=0.51, recall_weighted=0.76, precision_micro=0.76, precision_macro=0.54, precision_weighted=0.76


test loss=1.30: 100%|██████████| 3066/3066 [02:18<00:00, 22.17it/s]
/Users/aliyilmaz/anaconda3/envs/pytorch_env/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/aliyilmaz/anaconda3/envs/pytorch_env/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


...

test loss=1.69: 100%|██████████| 3066/3066 [01:42<00:00, 29.84it/s]
/Users/aliyilmaz/anaconda3/envs/pytorch_env/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/aliyilmaz/anaconda3/envs/pytorch_env/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


epoch 13: trainloss 1.73, testloss 0.82 accuracy=0.66, kappa=0.56, f1_micro=0.66, f1_macro=0.40, f1_weighted=0.65, recall_micro=0.66, recall_macro=0.39, recall_weighted=0.66, precision_micro=0.66, precision_macro=0.45, precision_weighted=0.67


test loss=1.75: 100%|██████████| 3066/3066 [01:53<00:00, 27.06it/s]


epoch 14: trainloss 1.67, testloss 0.57 accuracy=0.75, kappa=0.67, f1_micro=0.75, f1_macro=0.51, f1_weighted=0.72, recall_micro=0.75, recall_macro=0.52, recall_weighted=0.75, precision_micro=0.75, precision_macro=0.52, precision_weighted=0.74


/Users/aliyilmaz/anaconda3/envs/pytorch_env/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/aliyilmaz/anaconda3/envs/pytorch_env/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [16]:
device = "mps" if torch.backends.mps.is_available() else "cpu"
model = LSTM_DANN(input_size=13, hidden_size=64, num_layers=2, output_size=9, bidirectional=False, dropout=0.2, c_norm=True, num_domains=3).to(device)
train_dataloader, test_dataloader, meta = get_dataloader124(batchsize=32, num_workers=0, X=0.8, upperPerformance=False)
criterion_label = torch.nn.CrossEntropyLoss(reduction="mean")
criterion_domain = torch.nn.CrossEntropyLoss(reduction="mean")
optimizer = Adam(model.parameters(), 0.001, weight_decay=1e-6)
log = list()
n_epoch = 10
for epoch in range(n_epoch):
    """if epoch < 2:
        alpha = 0.0
    else:
        p = float(epoch-2) / (n_epoch-2) 
        #alpha = (2. / (1. + np.exp(-5 * p)) - 1) * 0.4 + 0.1
        alpha = 0.1 + 0.4 * p"""
    p = float(epoch) / n_epoch
    alpha = (2. / (1. + np.exp(-10 * p)) - 1)
    train_loss, label_loss, domain_loss = train_epoch(model, optimizer, criterion_label, criterion_domain, train_dataloader, device, alpha=alpha)
    test_loss, y_true, y_pred, *_ = test_epoch(model, criterion_label, test_dataloader, device)
    scores1 = metrics(y_true.cpu(), y_pred.cpu())
    scores_msg = ", ".join([f"{k}={v:.2f}" for (k, v) in scores1.items()])
    test_loss = test_loss.cpu().detach().numpy().mean()
    train_loss = train_loss.cpu().detach().numpy().mean()
    print(f"epoch {epoch}: trainloss {train_loss:.2f}, testloss {test_loss:.2f} " + scores_msg)

    scores1["epoch"] = epoch
    scores1["trainloss"] = train_loss
    scores1["testloss"] = test_loss
    scores1["labelloss"] = label_loss.cpu().detach().numpy().mean()
    scores1["domainloss"] = domain_loss.cpu().detach().numpy().mean()
    log.append(scores1)

    log_df = pd.DataFrame(log).set_index("epoch")
    log_df.to_csv("DANN124.csv")

test loss=1.19: 100%|██████████| 4160/4160 [03:35<00:00, 19.35it/s]
/Users/aliyilmaz/anaconda3/envs/pytorch_env/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/aliyilmaz/anaconda3/envs/pytorch_env/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


epoch 0: trainloss 0.72, testloss 0.60 accuracy=0.74, kappa=0.66, f1_micro=0.74, f1_macro=0.49, f1_weighted=0.73, recall_micro=0.74, recall_macro=0.49, recall_weighted=0.74, precision_micro=0.74, precision_macro=0.50, precision_weighted=0.73


test loss=1.99: 100%|██████████| 4160/4160 [03:34<00:00, 19.39it/s]
/Users/aliyilmaz/anaconda3/envs/pytorch_env/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/aliyilmaz/anaconda3/envs/pytorch_env/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


epoch 1: trainloss 1.63, testloss 0.62 accuracy=0.74, kappa=0.66, f1_micro=0.74, f1_macro=0.48, f1_weighted=0.71, recall_micro=0.74, recall_macro=0.52, recall_weighted=0.74, precision_micro=0.74, precision_macro=0.48, precision_weighted=0.73


test loss=0.89: 100%|██████████| 4160/4160 [03:37<00:00, 19.14it/s]
/Users/aliyilmaz/anaconda3/envs/pytorch_env/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/aliyilmaz/anaconda3/envs/pytorch_env/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


...

test loss=1.50: 100%|██████████| 4160/4160 [03:35<00:00, 19.26it/s]
/Users/aliyilmaz/anaconda3/envs/pytorch_env/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/aliyilmaz/anaconda3/envs/pytorch_env/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


epoch 13: trainloss 1.78, testloss 0.87 accuracy=0.67, kappa=0.56, f1_micro=0.67, f1_macro=0.42, f1_weighted=0.65, recall_micro=0.67, recall_macro=0.44, recall_weighted=0.67, precision_micro=0.67, precision_macro=0.42, precision_weighted=0.65


test loss=1.05: 100%|██████████| 4160/4160 [03:40<00:00, 18.89it/s]


epoch 14: trainloss 1.74, testloss 0.65 accuracy=0.73, kappa=0.63, f1_micro=0.73, f1_macro=0.48, f1_weighted=0.69, recall_micro=0.73, recall_macro=0.48, recall_weighted=0.73, precision_micro=0.73, precision_macro=0.49, precision_weighted=0.71


/Users/aliyilmaz/anaconda3/envs/pytorch_env/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/aliyilmaz/anaconda3/envs/pytorch_env/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [17]:
device = "mps" if torch.backends.mps.is_available() else "cpu"
model = LSTM_DANN(input_size=13, hidden_size=64, num_layers=2, output_size=9, bidirectional=False, dropout=0.2, c_norm=True, num_domains=3).to(device)
train_dataloader, test_dataloader, meta = get_dataloader134(batchsize=32, num_workers=0, X=0.8, upperPerformance=False)
criterion_label = torch.nn.CrossEntropyLoss(reduction="mean")
criterion_domain = torch.nn.CrossEntropyLoss(reduction="mean")
optimizer = Adam(model.parameters(), 0.001, weight_decay=1e-6)
log = list()
n_epoch = 15
for epoch in range(n_epoch):
    p = float(epoch) / n_epoch
    alpha = (2. / (1. + np.exp(-10 * p)) - 1)
    train_loss, label_loss, domain_loss = train_epoch(model, optimizer, criterion_label, criterion_domain, train_dataloader, device, alpha=alpha)
    test_loss, y_true, y_pred, *_ = test_epoch(model, criterion_label, test_dataloader, device)
    scores1 = metrics(y_true.cpu(), y_pred.cpu())
    scores_msg = ", ".join([f"{k}={v:.2f}" for (k, v) in scores1.items()])
    test_loss = test_loss.cpu().detach().numpy().mean()
    train_loss = train_loss.cpu().detach().numpy()[0].mean()
    print(f"epoch {epoch}: trainloss {train_loss:.2f}, testloss {test_loss:.2f} " + scores_msg)

    scores1["epoch"] = epoch
    scores1["trainloss"] = train_loss
    scores1["testloss"] = test_loss
    scores1["labelloss"] = label_loss.cpu().detach().numpy().mean()
    scores1["domainloss"] = domain_loss.cpu().detach().numpy().mean()
    log.append(scores1)

    log_df = pd.DataFrame(log).set_index("epoch")
    log_df.to_csv("DANN134.csv")

  0%|          | 0/14614 [00:00<?, ?it/s]

test loss=0.83: 100%|██████████| 3517/3517 [03:03<00:00, 19.14it/s]
/Users/aliyilmaz/anaconda3/envs/pytorch_env/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/aliyilmaz/anaconda3/envs/pytorch_env/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


epoch 0: trainloss 2.43, testloss 0.72 accuracy=0.67, kappa=0.56, f1_micro=0.67, f1_macro=0.44, f1_weighted=0.63, recall_micro=0.67, recall_macro=0.44, recall_weighted=0.67, precision_micro=0.67, precision_macro=0.50, precision_weighted=0.72


test loss=0.50: 100%|██████████| 3517/3517 [03:05<00:00, 18.96it/s]
/Users/aliyilmaz/anaconda3/envs/pytorch_env/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/aliyilmaz/anaconda3/envs/pytorch_env/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


epoch 1: trainloss 1.71, testloss 0.63 accuracy=0.72, kappa=0.63, f1_micro=0.72, f1_macro=0.50, f1_weighted=0.71, recall_micro=0.72, recall_macro=0.49, recall_weighted=0.72, precision_micro=0.72, precision_macro=0.55, precision_weighted=0.76


...

epoch 8: trainloss 1.68, testloss 0.51 accuracy=0.75, kappa=0.68, f1_micro=0.75, f1_macro=0.53, f1_weighted=0.73, recall_micro=0.75, recall_macro=0.53, recall_weighted=0.75, precision_micro=0.75, precision_macro=0.57, precision_weighted=0.78


test loss=0.82: 100%|██████████| 3517/3517 [03:01<00:00, 19.34it/s]
/Users/aliyilmaz/anaconda3/envs/pytorch_env/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/aliyilmaz/anaconda3/envs/pytorch_env/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


epoch 9: trainloss 1.55, testloss 0.49 accuracy=0.78, kappa=0.71, f1_micro=0.78, f1_macro=0.55, f1_weighted=0.77, recall_micro=0.78, recall_macro=0.55, recall_weighted=0.78, precision_micro=0.78, precision_macro=0.56, precision_weighted=0.78


train loss=0.34:  53%|█████▎    | 7710/14614 [07:39<06:51, 16.78it/s]


KeyboardInterrupt: 